In [1]:
from models import *
from metric_functions import *

import numpy as np
import pandas as pd

print("GPUs Available: ", tf.config.list_physical_devices('GPU'), flush=True)
physical_devices = tf.config.list_physical_devices('GPU')
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.set_visible_devices(physical_devices[1:], 'GPU')
#print("GPUs Available: ", tf.config.list_logical_devices('GPU'), flush=True)

2022-08-07 18:08:49.249409: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


GPUs Available:  []


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2022-08-07 18:08:51.486032: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-07 18:08:51.487034: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-07 18:08:51.501805: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-08-07 18:08:51.501841: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 8557ee92613c
2022-08-07 18:08:51.501850: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 8557ee92613c
2022-08-07 18:08:51.502019: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.39.1
2022-08-07 18:08:51.502039: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [2]:
# Define paths
USE_NA = False
if USE_NA:
    biobank_path = 'data/BioBank_NA.xlsx'
else:
    biobank_path = 'data/BioBank.xlsx'

save_folder = "models/"

In [3]:
# Load files
bb_train_df = pd.read_excel(biobank_path, sheet_name='Training Set', engine='openpyxl')
bb_test_df = pd.read_excel(biobank_path, sheet_name='Testing Set', engine='openpyxl')

# NOTE: there is also a full_overlap_data version in the twins h5 file, but
# here we are just concatenating the loaded train and test datasets, to
# preserve sample order
bb_full_data = pd.concat([bb_train_df, bb_test_df], ignore_index = True)

print('Twins data shape:\t'        + str(bb_full_data.shape))
print('Twins train data shape:\t'  + str(bb_train_df.shape))
print('Twins test data shape:\t'   + str(bb_test_df.shape))
print('')

Twins data shape:	(98698, 168)
Twins train data shape:	(83893, 168)
Twins test data shape:	(14805, 168)



In [4]:
bb_train_df

,p23400_i0,p23401_i0,p23402_i0,p23403_i0,p23404_i0,p23405_i0,p23406_i0,p23407_i0,p23408_i0,p23409_i0,...,p23569_i0,p23570_i0,p23571_i0,p23572_i0,p23573_i0,p23574_i0,p23575_i0,p23576_i0,p23577_i0,p23578_i0
0,1.370488,0.677570,0.765376,-0.642959,0.785040,0.396673,1.121010,-1.313618,-1.301146,-1.403214,...,1.087412,1.171302,-0.034900,0.641719,0.627415,0.700525,0.697437,0.663621,0.692945,-0.957609
1,1.131893,-0.295819,-0.379010,-1.648618,-0.109481,-0.121048,1.583286,-1.092159,-1.407170,0.426890,...,1.159730,1.321750,-0.283059,-0.111369,-0.040375,0.046780,0.041504,-0.155049,0.801762,-0.966301
2,0.587712,0.269678,-0.134466,-0.438271,0.461926,0.432414,0.588021,-0.673478,-0.588343,-0.453330,...,0.429221,0.457475,-0.771196,0.064433,-0.164262,-0.262105,0.150935,0.153376,0.131241,-0.739312
3,-0.308820,0.961742,0.730830,1.303594,0.818278,0.776691,-1.629694,0.890498,0.950183,0.809907,...,-1.473580,-1.532490,-0.267726,-0.650874,-0.914805,-1.093708,-0.737750,-0.635374,-0.951352,0.504762
4,-0.969326,-0.862400,-0.882208,-0.117285,-0.783888,-0.523591,-0.218689,1.059166,1.011914,1.128079,...,-0.153356,-0.218180,0.709359,0.094501,0.234631,0.221087,0.073170,0.071763,0.087750,0.954576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83888,-0.786897,0.670984,1.395157,2.236662,0.055445,-0.047459,-2.058299,1.782279,1.707419,1.306704,...,-1.954260,-1.729018,0.987142,-1.305954,-1.110451,-1.062773,-1.502338,-1.400323,-1.544480,1.214798
83889,1.370888,-0.407966,-0.578481,-1.131710,-0.437105,-0.140350,1.868623,0.067840,0.129923,-0.115701,...,1.604690,1.721331,-0.060757,0.918604,1.073881,1.162624,1.056794,0.900948,1.462472,-0.661418
83890,-1.287074,-0.073450,0.751586,2.097656,-0.836363,-0.597922,-1.697697,2.221443,2.067491,2.121856,...,-1.727360,-1.266037,1.204901,-1.716239,-1.199421,-1.026194,-1.896726,-1.930883,-1.376811,1.442483
83891,0.205896,0.324734,0.121870,0.153390,0.370858,0.350230,0.064260,-0.612404,-0.508697,-0.394281,...,-0.050643,-0.125595,-1.028958,-0.211028,-0.390524,-0.410977,-0.190956,-0.120405,-0.408009,-0.651064


In [5]:
# Data & model configuration
input_dim = bb_train_df.shape[1]
params = load_vae_parameters(optimal=False)

intermediate_dim = max(params['encoder_units'], params['decoder_units'])
latent_dim = 18

kl_beta = params['kl_beta']
learning_rate = params['learning_rate']

batch_size = 256
n_epochs = 1000


# instantiate model
mtmodel = mtVAE(input_dim,
                intermediate_dim,
                latent_dim,
                kl_beta,
                learning_rate)

In [6]:
# Train model
mtmodel.train(bb_train_df, bb_test_df, n_epochs, batch_size)

Train on 83893 samples, validate on 14805 samples


2022-08-07 18:12:46.093475: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-07 18:12:46.106426: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-07 18:12:46.276948: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2022-08-07 18:12:46.285655: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2749780000 Hz


Epoch 1/1000


/opt/conda/envs/mtvae_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


83893/83893 - 3s - loss: 23.4638 - nan_safe_mse: 0.1354 - val_loss: 18.1289 - val_nan_safe_mse: 0.1034
Epoch 2/1000
83893/83893 - 3s - loss: 9.8978 - nan_safe_mse: 0.0546 - val_loss: 16.0217 - val_nan_safe_mse: 0.0910
Epoch 3/1000
83893/83893 - 3s - loss: 8.7832 - nan_safe_mse: 0.0481 - val_loss: 15.2390 - val_nan_safe_mse: 0.0865
Epoch 4/1000
83893/83893 - 3s - loss: 8.2433 - nan_safe_mse: 0.0449 - val_loss: 14.4209 - val_nan_safe_mse: 0.0817
Epoch 5/1000
83893/83893 - 2s - loss: 388.2721 - nan_safe_mse: 2.3052 - val_loss: 13.9217 - val_nan_safe_mse: 0.0788
Epoch 6/1000
83893/83893 - 3s - loss: 7.5647 - nan_safe_mse: 0.0410 - val_loss: 13.3098 - val_nan_safe_mse: 0.0752
Epoch 7/1000
83893/83893 - 3s - loss: 7.3838 - nan_safe_mse: 0.0399 - val_loss: 13.1723 - val_nan_safe_mse: 0.0744
Epoch 8/1000
83893/83893 - 3s - loss: 7.2168 - nan_safe_mse: 0.0390 - val_loss: 12.6728 - val_nan_safe_mse: 0.0714
Epoch 9/1000
83893/83893 - 4s - loss: 7.0700 - nan_safe_mse: 0.0381 - val_loss: 12.5070 - 

In [7]:
# Save model
mtmodel.save_model(save_folder)

## Calculate model performance metrics

In [8]:
# these arrays are used for score calculations
bb_train = bb_train_df.values
bb_test  = bb_test_df.values

In [9]:
######################
# Define PCA model
######################
if not USE_NA:
    PCA_model_ = PCA_model(bb_train, latent_dim)

In [10]:
# Generate TwinsUK dataset reconstructions with VAE and PCA
VAE_train = mtmodel.reconstruct(bb_train)
VAE_test  = mtmodel.reconstruct(bb_test)

if not USE_NA:
    PCA_train = PCA_model_.reconstruct(bb_train)
    PCA_test  = PCA_model_.reconstruct(bb_test)

In [15]:
# Calculate sample-wise MSEs
test_mses       = np.mean(get_mse(VAE_test,  bb_test))
train_mses      = np.mean(get_mse(VAE_train, bb_train))

if not USE_NA:
    test_mses_pca   = np.mean(get_mse(PCA_test,  bb_test))
    train_mses_pca  = np.mean(get_mse(PCA_train, bb_train))
else:
    test_mses_pca   = "NA"
    train_mses_pca  = "NA"


print('==================== Sample-wise ============================')
print('======================= MSEs ================================')
print('\t \t \t VAE \t \t \t PCA')
print('Twins train:\t ',  train_mses, '\t ', train_mses_pca)
print('Twins test:\t ',   test_mses,  '\t ', test_mses_pca)
print('')
print('')


# Calculate correlation matrix MSEs (i.e. CM-MSEs)
test_mses  = matrix_mse(VAE_test,  bb_test)
train_mses = matrix_mse(VAE_train, bb_train)

if not USE_NA:
    test_mses_pca  = matrix_mse(PCA_test,  bb_test)
    train_mses_pca = matrix_mse(PCA_train, bb_train)
else:
    test_mses_pca  = "NA"
    train_mses_pca = "NA"


print('================= Correlation Matrix ========================')
print('======================= MSEs ================================')
print('\t \t \t VAE \t \t \t PCA')
print('Twins train:\t ',  train_mses, '\t ', train_mses_pca)
print('Twins test:\t ',   test_mses, '\t ',  test_mses_pca)

==================== Sample-wise ============================
======================= MSEs ================================
	 	 	 VAE 	 	 	 PCA
Twins train:	  0.020709489269205857 	  0.060802188146953484
Twins test:	  0.026721106142330957 	  0.06807224488924754


================= Correlation Matrix ========================
======================= MSEs ================================
	 	 	 VAE 	 	 	 PCA
Twins train:	  0.00024987698966502577 	  0.0013920162280750157
Twins test:	  0.0005494100809716971 	  0.0014831884546800131
